# Environment

In [1]:
from optionlib.data import ticker, prices, earnings
from optionlib import *

In [2]:
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar   
import datetime as dt
from joblib import Parallel, delayed

# CSV to parquet conversion

## Option chain

In [7]:
%%time
for d in range(2012,2025):
    
    files = [i for i in os.listdir('../CBOE raw data/order_000050203/item_000057965') if str(d) in i]
    
    pd.concat(
        [pd.read_csv(f'../CBOE raw data/order_000050203/item_000057965/{f}',encoding='utf-8') for f in files]
    ).assign(
        quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
        expiration = lambda x: pd.to_datetime(x.expiration).dt.date
    ).to_parquet(f'../historical_data/spx_option_chain_historical_{d}.parquet')

CPU times: user 8min 13s, sys: 1min 43s, total: 9min 56s
Wall time: 10min 20s


## Price data

In [99]:
prices_raw = pd.concat(
    [pd.read_csv(f'../CBOE raw data/order_000050263/item_000058027/{f}') 
     for f in os.listdir('../CBOE raw data/order_000050263/item_000058027')]
).to_parquet('../historical_data/spx_price_history.parquet')

In [ ]:
tbill_1mo_url_csv = 'https://fred.stlouisfed.org/graph/fredgraph.csv?mode=fred&id=DGS1MO&vintage_date=2024-01-13&revision_date=2024-01-13&nd=1954-07-01'

tbill = pd.read_csv(tbill_1mo_url_csv).rename(
    columns={'DATE':'Date'}
).assign(
    Date = lambda x: pd.to_datetime(x.Date)
).set_index('Date').replace('.',np.nan).astype(float).ffill()

In [ ]:
prices = prices.assign(
    quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
    Date = lambda x: x.quote_datetime.dt.date
).set_index(['quote_datetime','Date'],drop = False).join(tbill)

In [148]:
prices.to_parquet('../historical_data/spx_price_history.parquet')

# Calculations

## Implied volatility Black Scholes function

In [2]:
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)    
    

def implied_vol(opt_value, S, K, T, r, type_='C'):
    
    def call_obj(sigma):
        return abs(BS_CALL(S, K, T, r, sigma) - opt_value)
    
    def put_obj(sigma):
        return abs(BS_PUT(S, K, T, r, sigma) - opt_value)
    
    if type_ == 'C':
        res = minimize_scalar(call_obj, bounds=(0.01,6), method='bounded')
        return res.x
    elif type_ == 'P':
        res = minimize_scalar(put_obj, bounds=(0.01,6),
                              method='bounded')
        return res.x
    else:
        raise ValueError("type_ must be 'put' or 'call'")

## For a single year

In [5]:
max_T = 30
strike_window = 0.05

oc_2024 = pd.read_parquet('../historical_data/spx_option_chain_historical_2024.parquet')
prices = pd.read_parquet('../historical_data/spx_price_history.parquet')

oc_2024 = oc_2024.assign(
    quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
    expiration = lambda x: pd.to_datetime(x.expiration) + np.timedelta64(975,'m'),
    time_to_expiry = lambda x: (x.expiration - x.quote_datetime).div(np.timedelta64(1,'D')).astype(float),
    bs_time = lambda x: x.time_to_expiry.div(365)
).set_index(['quote_datetime','expiration','strike','option_type'],drop = False)

oc_2024 = oc_2024.join(prices.rename(columns = {'close':'SPX_open'})[['SPX_open','DGS1MO']])

oc_2024_filtered = oc_2024.loc[
    lambda x: x.open.gt(0)
    & x.bs_time.gt(0)
    & x.time_to_expiry.le(7)
    & x.strike.between(x.SPX_open*(1-strike_window),x.SPX_open*(1+strike_window))
]

In [6]:
%%time
spx_iv = oc_2024_filtered.apply(
    lambda x: implied_vol(
        x.open,
        x.SPX_open,
        x.strike,
        x.bs_time,
        x.DGS1MO,
        x.option_type
    ),axis = 1
)

CPU times: user 1min 26s, sys: 114 ms, total: 1min 27s
Wall time: 1min 27s


## For all years

In [13]:
%%time

max_T = 30
strike_window = 0.05

prices = pd.read_parquet('../historical_data/spx_price_history.parquet')

def process_IV(y):
    oc_temp = pd.read_parquet(f'../historical_data/spx_option_chain_historical_{y}.parquet')

    oc_temp = oc_temp.assign(
        quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
        expiration = lambda x: pd.to_datetime(x.expiration) + np.timedelta64(975,'m'),
        time_to_expiry = lambda x: (x.expiration - x.quote_datetime).div(np.timedelta64(1,'D')).astype(float),
        bs_time = lambda x: x.time_to_expiry.div(365)
    ).set_index(['quote_datetime','expiration','strike','option_type'],drop = False)

    oc_temp = oc_temp.join(prices.rename(columns = {'close':'SPX_open'})[['SPX_open','DGS1MO']])

    oc_temp = oc_temp.loc[
        lambda x: x.open.gt(0)
        & x.bs_time.gt(0)
        & x.time_to_expiry.le(max_T)
        & x.strike.between(x.SPX_open*(1-strike_window),x.SPX_open*(1+strike_window))
    ]
    
    oc_out = oc_temp.apply(
        lambda x: implied_vol(x.open,x.SPX_open,x.strike,x.bs_time,0.05,x.option_type),
        axis = 1
    )
    
    oc_out.rename('IV').to_frame().to_parquet(f'../historical_data/spx_iv_{y}_backup.parquet')
    
    return oc_out

IV_dict = Parallel(n_jobs=4,verbose = 5)(delayed(process_IV)(i) for i in range(2012,2025))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done   9 out of  13 | elapsed: 32.7min remaining: 14.6min
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 291 ms, sys: 375 ms, total: 666 ms
Wall time: 1h 1min 54s


[Parallel(n_jobs=4)]: Done  13 out of  13 | elapsed: 61.9min finished


In [14]:
%%time

max_T = 30
strike_window = 0.05

prices = pd.read_parquet('../historical_data/spx_price_history.parquet')

def process_IV(y):
    oc_temp = pd.read_parquet(f'../historical_data/spx_option_chain_historical_{y}.parquet')

    oc_temp = oc_temp.assign(
        quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
        expiration = lambda x: pd.to_datetime(x.expiration) + np.timedelta64(975,'m'),
        time_to_expiry = lambda x: (x.expiration - x.quote_datetime).div(np.timedelta64(1,'D')).astype(float),
        bs_time = lambda x: x.time_to_expiry.div(365)
    ).set_index(['quote_datetime','expiration','strike','option_type'],drop = False)

    oc_temp = oc_temp.join(prices.rename(columns = {'close':'SPX_open'})[['SPX_open','DGS1MO']])

    oc_temp = oc_temp.loc[
        lambda x: x.open.gt(0)
        & x.bs_time.gt(0)
        & x.time_to_expiry.le(max_T)
        & x.strike.between(x.SPX_open*(1-strike_window),x.SPX_open*(1+strike_window))
    ]
    
    oc_out = oc_temp.apply(
        lambda x: implied_vol(x.open,x.SPX_open,x.strike,x.bs_time,0.05,x.option_type),
        axis = 1
    )
    
    oc_out.rename('IV').to_frame().to_parquet(f'../historical_data/spx_iv_{y}_backup_30d.parquet')
    
    return oc_out

IV_dict = Parallel(n_jobs=4,verbose = 5)(delayed(process_IV)(i) for i in range(2012,2025))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done   9 out of  13 | elapsed: 86.3min remaining: 38.3min


CPU times: user 517 ms, sys: 588 ms, total: 1.11 s
Wall time: 2h 44min 23s


[Parallel(n_jobs=4)]: Done  13 out of  13 | elapsed: 164.4min finished


# Backtesting

In [ ]:
y = 2023
oc = pd.read_parquet(f'../historical_data/spx_option_chain_historical_{y}.parquet')
adj_factor_hist = pd.read_csv('../horizon_5_adj_factor.csv')
adj_factor_hist_aug = pd.read_csv('../horizon_5_adj_factor.csv',index_col=0)

oc = oc.assign(
    quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
    expiration = lambda x: pd.to_datetime(x.expiration).dt.date,
    days_to_expiry = lambda x: (x.expiration - x.quote_datetime.dt.date).div(np.timedelta64(1,'D')).astype(int),
    midpoint = lambda x: round((x.bid+x.ask)/2,2),
    put_call = lambda x: x.option_type
).rename(
    columns = lambda x: x.capitalize()
).rename(
    columns = {'Option_type':'put_call'}
).set_index(['Quote_datetime','put_call','Strike'],drop = False)

oc = oc.loc[
    oc.Quote_datetime.dt.dayofweek.eq(0)
    & oc.Days_to_expiry.eq(4)
    & oc.Quote_datetime.dt.time.eq(dt.time(hour = 10, minute = 30)),
    ['Bid','Ask']
]

# data = ticker.base_data(risk_ticker='SPY',)

In [46]:
adj_factor_hist = pd.read_csv('../horizon_5_adj_factor.csv')
adj_factor_hist_aug = pd.read_csv('../horizon_5_adj_factor.csv',index_col=0)

In [63]:
dt_str = str(dt.date())

In [64]:
model_obj = model.Model(
        data.fillna(method = 'pad'),
        horizon = 5,
        eval_date = dt_str
    )

/var/folders/5p/b_k1m3qs09q0s_768gjrmvvh0000gp/T/ipykernel_98357/3154462887.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method = 'pad'),


In [70]:
var_list = '|'.join([
            'pct_delta_\d',
            'shiller'
            'T1',
            'VIX',
            'hv',
            'Volume',
            'jobs_friday',
            'month'
        ])

In [73]:
model_obj.data.index.get_loc(dt_str)

7536

In [76]:
model_obj.data.dropna().filter(regex = var_list).iloc[
                :model_obj.data.dropna().index.get_loc(dt_str) - 5,:
                ]

,Volume,Volume_7d,pct_delta_1d,pct_delta_2d,pct_delta_3d,pct_delta_4d,pct_delta_5d,pct_delta_6d,pct_delta_7d,pct_delta_30d,pct_delta_60d,pct_delta_90d,pct_delta_180d,pct_delta_360d,hv30,VIX,jobs_friday,month
Date,,,,,,,,,,,,,,,,,,
1994-07-05,112000,1.029986e+06,0.005259,0.007379,0.001047,0.004203,-0.000697,0.016667,-0.004514,-0.016804,-0.005205,-0.051291,-0.048141,0.019559,0.497505,14.920000,1.0,7
1994-07-06,174800,1.004414e+06,-0.001395,0.003857,0.005973,-0.000349,0.002802,-0.002091,0.015248,-0.016489,0.001049,-0.039906,-0.046937,0.010946,0.497281,14.700000,1.0,7
1994-07-07,66700,9.609143e+05,0.004191,0.002790,0.008065,0.010190,0.003841,0.007005,0.002091,-0.016421,0.001045,-0.040387,-0.035882,0.013037,0.497504,14.010000,1.0,7
1994-07-08,148400,2.132143e+05,-0.000348,0.003842,0.002442,0.007714,0.009838,0.003492,0.006655,-0.019447,0.002092,-0.040721,-0.037508,0.002092,0.493731,13.260000,0.0,7
1994-07-11,124000,1.863857e+05,-0.003479,-0.003826,0.000349,-0.001046,0.004208,0.006325,0.000000,-0.023525,0.003856,-0.040214,-0.039571,-0.005556,0.495736,13.830000,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-16,119858000,9.814987e+07,-0.016323,-0.040386,-0.046521,-0.039303,-0.025453,-0.032733,-0.025155,0.033045,0.024184,-0.087431,-0.151382,-0.118413,2.539201,22.620001,0.0,12
2022-12-19,79878100,1.008842e+08,-0.008480,-0.024664,-0.048523,-0.054606,-0.047450,-0.033716,-0.040935,0.009752,0.032803,-0.095169,-0.160956,-0.118978,2.505561,22.420000,0.0,12
2022-12-20,74427200,9.988126e+07,0.001368,-0.007123,-0.023330,-0.047221,-0.053312,-0.046146,-0.032394,0.001553,0.044550,-0.109014,-0.166944,-0.104549,2.476386,21.480000,0.0,12


In [69]:
model_obj.y

,pct_delta_ahead
Date,
1994-07-05,0.000349
1994-07-06,0.003493
1994-07-07,0.010087
1994-07-08,0.010786
1994-07-11,0.016061
...,...
2024-01-04,0.019410
2024-01-05,0.018721
2024-01-08,0.000695


In [55]:
trades = dict()
trade_quantiles = dict()

for dt in oc.Quote_datetime.sort_values().unique()[:5]:
    dt_str = str(dt.date())
    
    model_obj = model.Model(
        data.fillna(method = 'pad'),
        horizon = 5,
        eval_date = dt_str
    )
    
    floating_quintile = floating_quantile.floating_quantile(
        adj_factor = adj_factor_hist_aug.loc[dt_str].values[0],
        y = model_obj.y,
        adj_factor_hist = adj_factor_hist
    )

    bounds = tuple(round(i) for i in floating_quintile.agg(['min','max']).add(1).multiply(data.Close[-1]))

    menu = menus.TradeMenu(
        oc.loc[dt,['Bid','Ask']].sort_index(),#.iloc[[i for i in range(0,len(prices_df),2)]],
        data.Close[-1],
        floating_quintile,
        bounds = bounds,
        # midpoint_price = True
    )

    max_EV_harmonic_idx = menu.menu.loc[lambda x: x.win_pct.between(0.6,0.9)].EV_harmonic.idxmax()
    trades[dt] = menu.menu.loc[max_EV_harmonic_idx]
    trade_quantiles[dt] = menu.quantiles.loc[max_EV_harmonic_idx]

/var/folders/5p/b_k1m3qs09q0s_768gjrmvvh0000gp/T/ipykernel_98357/106692077.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method = 'pad'),
/var/folders/5p/b_k1m3qs09q0s_768gjrmvvh0000gp/T/ipykernel_98357/106692077.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bounds = tuple(round(i) for i in floating_quintile.agg(['min','max']).add(1).multiply(data.Close[-1]))
/var/folders/5p/b_k1m3qs09q0s_768gjrmvvh0000gp/T/ipykernel_98357/106692077.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.Close[-1],


Write strategies complete
Spreads and straddles complete
Calculating 0 Iron Condors...
Calculating 0 Reverse Iron Condors...
Iron condors complete. Transforming payout quantiles...
Calculating payout characteristics


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   0 out of   0 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   0 out of   0 | elapsed:    0.0s finished
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/optionlib/menus.py:274: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  kelly_criteria_EV_harmonic = self.EV_harmonic.idxmax(axis = 1)


ValueError: attempt to get argmax of an empty sequence